In [1]:
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import warnings
warnings.filterwarnings('ignore')

In [4]:
# define the parameters of the dataset
custom_ds_metadata = {
    'num_series': 100,
    'num_steps': 24 * 7,
    'prediction_length': 24,
    'freq': '1H',
    'start': [
        pd.Timestamp("01-01-2019", freq='1H')
        for _ in range(100)
    ]
}

In [21]:
# pattern - sinusoid with different phase
period = 4
mu = 1
sigma = 3
num_steps = 4*7
num_series = 2
sin_minusPi_Pi = np.sin(np.tile(np.linspace(-np.pi, np.pi, period), int(num_steps / period)))
sin_Zero_2Pi = np.sin(np.tile(np.linspace(0, 2 * np.pi, 4), int(num_steps / period)))

In [22]:
sin_minusPi_Pi

array([-1.22464680e-16, -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
       -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
       -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
       -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
       -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
       -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
       -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,  1.22464680e-16])

In [23]:
sin_Zero_2Pi

array([ 0.00000000e+00,  8.66025404e-01, -8.66025404e-01, -2.44929360e-16,
        0.00000000e+00,  8.66025404e-01, -8.66025404e-01, -2.44929360e-16,
        0.00000000e+00,  8.66025404e-01, -8.66025404e-01, -2.44929360e-16,
        0.00000000e+00,  8.66025404e-01, -8.66025404e-01, -2.44929360e-16,
        0.00000000e+00,  8.66025404e-01, -8.66025404e-01, -2.44929360e-16,
        0.00000000e+00,  8.66025404e-01, -8.66025404e-01, -2.44929360e-16,
        0.00000000e+00,  8.66025404e-01, -8.66025404e-01, -2.44929360e-16])

In [24]:
noise = np.random.normal(mu, sigma, size=(num_series, num_steps))

In [25]:
pattern = np.concatenate(
        (
            np.tile(
                sin_minusPi_Pi.reshape(1, -1),
                (int(np.ceil(num_series / 2)),1)
            ),
            np.tile(
                sin_Zero_2Pi.reshape(1, -1),
                (int(np.floor(num_series / 2)), 1)
            )
        ),
        axis=0
    )

In [26]:
pattern

array([[-1.22464680e-16, -8.66025404e-01,  8.66025404e-01,
         1.22464680e-16, -1.22464680e-16, -8.66025404e-01,
         8.66025404e-01,  1.22464680e-16, -1.22464680e-16,
        -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
        -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,
         1.22464680e-16, -1.22464680e-16, -8.66025404e-01,
         8.66025404e-01,  1.22464680e-16, -1.22464680e-16,
        -8.66025404e-01,  8.66025404e-01,  1.22464680e-16,
        -1.22464680e-16, -8.66025404e-01,  8.66025404e-01,
         1.22464680e-16],
       [ 0.00000000e+00,  8.66025404e-01, -8.66025404e-01,
        -2.44929360e-16,  0.00000000e+00,  8.66025404e-01,
        -8.66025404e-01, -2.44929360e-16,  0.00000000e+00,
         8.66025404e-01, -8.66025404e-01, -2.44929360e-16,
         0.00000000e+00,  8.66025404e-01, -8.66025404e-01,
        -2.44929360e-16,  0.00000000e+00,  8.66025404e-01,
        -8.66025404e-01, -2.44929360e-16,  0.00000000e+00,
         8.66025404e-01, -8.66

In [27]:
target = noise + pattern

In [28]:
target

array([[ 2.09010763, -0.41318195,  5.00762719,  4.33069581, -1.40095061,
         2.60246174, -1.63097461,  0.17736774,  1.0540663 , -7.20620816,
        -1.88612126,  0.29748081, -5.44516384, -1.75954775,  2.87367922,
        -3.7996369 ,  1.11990647, -0.65606992, -0.64906578,  3.23278006,
         0.15838167, -0.06088533, 11.35859876, -1.02416723,  6.1449854 ,
         0.70585489,  3.24955491,  0.49726386],
       [ 4.71347304,  3.83775038,  5.01899694, -1.79945097,  2.80885843,
         0.65856029, -2.95057702, -0.63742993,  0.31825404,  0.46037696,
         1.88070498,  4.58400082,  1.62283974,  4.34965186,  0.29975393,
         4.64840522, -1.90266333,  0.65416618, -4.01414465,  0.03833083,
         5.66436101,  0.04992216,  1.35743375,  5.53865319,  1.82035351,
         1.48393504,  6.77843815,  3.47795146]])

In [29]:
feat_dynamic_real = np.concatenate(
    (
        np.zeros((num_series, period)),
        target[:, :-period]
    ),
    axis=1
)

# create categorical static feats: use the sinusoid type as a categorical feature
feat_static_cat = np.concatenate(
    (
        np.zeros(int(np.ceil(num_series / 2))),
        np.ones(int(np.floor(num_series / 2)))
    ),
    axis=0
)

In [30]:
feat_dynamic_real

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  2.09010763,
        -0.41318195,  5.00762719,  4.33069581, -1.40095061,  2.60246174,
        -1.63097461,  0.17736774,  1.0540663 , -7.20620816, -1.88612126,
         0.29748081, -5.44516384, -1.75954775,  2.87367922, -3.7996369 ,
         1.11990647, -0.65606992, -0.64906578,  3.23278006,  0.15838167,
        -0.06088533, 11.35859876, -1.02416723],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  4.71347304,
         3.83775038,  5.01899694, -1.79945097,  2.80885843,  0.65856029,
        -2.95057702, -0.63742993,  0.31825404,  0.46037696,  1.88070498,
         4.58400082,  1.62283974,  4.34965186,  0.29975393,  4.64840522,
        -1.90266333,  0.65416618, -4.01414465,  0.03833083,  5.66436101,
         0.04992216,  1.35743375,  5.53865319]])

In [31]:
feat_static_cat

array([0., 1.])